In [13]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ce40e9b043decde533bad6a9c9cda7864e3cbfa12fe6b76ba30c012f608775e7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
!pip install fuzzywuzzy

In [6]:
import pandas as pd
relation_df = pd.read_csv("/content/relation_extraction_rebel.csv",index_col=0)
relation_df.head()

,Subject,Relation,Object
0,action,part of,strategy
1,risk order,studied by,risk management
2,risk response,studied by,risk management
3,pmi,instance of,standard
4,comparable situation,subclass of,historical review


In [3]:
import pandas as pd
df = pd.read_csv("/content/PMI_PMBOK_Definitions.csv")
df.head()

,Concept,Cleaned_Description,References,Sentences,KeyWords_KeyBert,Synonyms_SBert,Description,Definition
0,PROJECT RISK MANAGEMENT,project risk management includes process condu...,['https://i.ibb.co/pWPHv3r/396.png'],['project risk management includes process con...,"['project risk managementall', 'practice proje...","['project risk managementall', 'practice proje...",Project Risk Management includes the processes...,Project Risk Management includes the processe...
1,Plan Risk Management,plan risk management process defining conduct ...,['https://i.ibb.co/rssfq3x/401.jpg'],['plan risk management process defining conduc...,"['risk management process requires', 'project ...","['plan risk management process', 'plan risk ma...",Plan Risk Management is the process of definin...,Plan Risk Management is the process of defini...
2,Plan Risk Management Inputs,11.1.1.1 project charter described section 4.1...,"['https://i.ibb.co/fXjdfx5/402.jpg', '4.1.3.1'...",['11.1.1.1 project charter described section 4...,"['responsibility managing risk project', 'risk...",[],11.1.1.1 PROJECT CHARTER Described in Section ...,The project charter documents the high-level ...
3,Plan Risk Management Tools and Techniques,11.1.2.1 expert judgment described section 4.1...,"['4.1.2.1', '13.1.2.3', '4.1.2.1', '13.1.2.3']",['11.1.2.1 expert judgment described section 4...,"['risk management specific need', 'conducting ...",['risk management plan developed'],11.1.2.1 EXPERT JUDGMENT Described in Section...,Expertise should be considered from individua...
4,Plan Risk Management Outputs,11.1.3.1 risk management planthe risk manageme...,"['https://i.ibb.co/RH2d7J4/406.png', 'https://...",['11.1.3.1 risk management planthe risk manage...,"['required project risk management', 'project ...",[],11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,The risk management plan is a component of th...


In [5]:
def extract_from_first_word(text):
    words = text.split()
    substring = ' '.join(words[words.index(next(filter(str.strip, words))):])
    return substring

# Apply the function to the DataFrame column
relation_df['Subject'] = relation_df['Subject'].apply(lambda x: extract_from_first_word(x))

In [14]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embedding for both lists
embeddings1 = model.encode(relation_df["Object"].tolist(), convert_to_tensor=True)
embeddings2 = model.encode(df["Concept"].tolist(), convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# For each 'Subject' in relation_df, find the index of the maximum similarity score
closest_concepts = []
for i in range(len(relation_df["Object"])):
    max_index = int(np.argmax(cosine_scores[i]))
    closest_concepts.append(df["Concept"].iloc[max_index])

# Update 'Subject' with the closest 'Concept'
relation_df['Object'] = closest_concepts


In [16]:
count_equal_subject_object = len(relation_df[relation_df["Subject"] == relation_df["Object"]])
print("Number of occurrences where Subject is equal to Object:", count_equal_subject_object)


Number of occurrences where Subject is equal to Object: 264


In [ ]:
relation_df = relation_df[relation_df["Subject"] != relation_df["Object"]]

In [19]:
relation_df.head(20)

,Subject,Relation,Object
0,Action Planning,part of,Develop Strategies before Tactical Responses
1,Risk Prioritization and Selection Guidelines,studied by,Good Risk Management Practice
2,Risk Response Strategies,studied by,Good Risk Management Practice
4,Scenario Analysis,subclass of,Historical Review
6,Conclusion,subclass of,REFERENCES
7,Risk Statement Metalanguage,facet of,Identify Risks
8,Categorize Risk Causes,subclass of,INTRODUCTION
9,Appropriate Project Model,subclass of,"Comply with the Organizations Objectives, Poli..."
10,"Provide Resources, Budget, and Schedule for Re...",subclass of,Monte Carlo Analysis of Finish Dates in Quanti...
11,Risk Response Strategies,part of,Plan Risk Management


In [20]:
relation_df.to_csv("relation_df2.csv",index=False)

In [24]:
relation_df2 = relation_df.copy()

In [ ]:
for idx, subject in relation_df2["Subject"].items():
    if subject in df["Concept"].values:
        concept_index = df.index[df["Concept"] == subject].tolist()[0]
        for col in ["Description", "Definition", "KeyWords_KeyBert", "Synonyms_SBert", "References"]:
            relation_df2.at[idx, f"Subject_{col}"] = df.at[concept_index, col]


In [32]:
for idx, subject in relation_df2["Object"].items():
    if subject in df["Concept"].values:
        concept_index = df.index[df["Concept"] == subject].tolist()[0]
        for col in ["Description", "Definition", "KeyWords_KeyBert", "Synonyms_SBert", "References"]:
            relation_df2.at[idx, f"Object_{col}"] = df.at[concept_index, col]


In [33]:
relation_df2.head()

,Subject,Relation,Object,Subject_Description,Subject_Definition,Subject_KeyWords_KeyBert,Subject_Synonyms_SBert,Subject_References,Object_Description,Object_Definition,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References
0,Action Planning,part of,Develop Strategies before Tactical Responses,Project planning tools are used to turn the ch...,[{'summary_text': ' Project planning tools are...,"['project planning', 'planning tool used', 'pr...",[],[],Risk response planning should be carried out i...,[{'summary_text': ' Risk response planning sho...,"['response planning carried', 'response planni...",[],[]
1,Risk Prioritization and Selection Guidelines,studied by,Good Risk Management Practice,The selection and prioritization of risks mus...,[{'summary_text': ' The selection and prioriti...,"['project objectives risk management', 'priori...","['prioritization risk', 'selection prioritizat...",[],Project Risk Management is a valuable componen...,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[]
2,Risk Response Strategies,studied by,Good Risk Management Practice,The project manager should develop risk respon...,[{'summary_text': ' The project manager should...,"['project level risks', 'develop risk response...","['develop risk response strategy', 'risk respo...",[],Project Risk Management is a valuable componen...,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[]
4,Scenario Analysis,subclass of,Historical Review,Scenario analysis for risk response planning ...,Scenario analysis for risk response planning ...,"['analysis risk response planning', 'scenario ...","['scenario analysis', 'scenario analysis risk']",[],Historical reviews are based on what occurred ...,Historical reviews are based on what occurred...,"['project organization comparable project', 'b...",[],[]
6,Conclusion,subclass of,REFERENCES,The principles of Project Risk Management desc...,The principles of Project Risk Management des...,"['project risk management provides', 'project ...",[],[],Association for Project Management. 2004. Proj...,Project risk analysis management PRAM guide s...,"['2004 project risk analysis', 'management 200...",[],[]


In [35]:
import ast
l = []
for desc in relation_df2["Subject_Definition"]:
    try:
        for j in ast.literal_eval(desc):
            l.append(j['summary_text'])
    except Exception as e:
        l.append(desc)
relation_df2.drop("Subject_Definition",inplace=True,axis=1)
relation_df2["Subject_Definition"] = l

In [36]:
import ast
l = []
for desc in relation_df2["Object_Definition"]:
    try:
        for j in ast.literal_eval(desc):
            l.append(j['summary_text'])
    except Exception as e:
        l.append(desc)
relation_df2.drop("Object_Definition",inplace=True,axis=1)
relation_df2["Object_Definition"] = l

In [37]:
relation_df2.head()

,Subject,Relation,Object,Subject_Description,Subject_KeyWords_KeyBert,Subject_Synonyms_SBert,Subject_References,Object_Description,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References,Subject_Definition,Object_Definition
0,Action Planning,part of,Develop Strategies before Tactical Responses,Project planning tools are used to turn the ch...,"['project planning', 'planning tool used', 'pr...",[],[],Risk response planning should be carried out i...,"['response planning carried', 'response planni...",[],[],Project planning tools are used to turn the c...,Risk response planning should be carried out ...
1,Risk Prioritization and Selection Guidelines,studied by,Good Risk Management Practice,The selection and prioritization of risks mus...,"['project objectives risk management', 'priori...","['prioritization risk', 'selection prioritizat...",[],Project Risk Management is a valuable componen...,"['project risk management general', 'project r...",[],[],The selection and prioritization of risks mus...,Project Risk Management is a valuable compone...
2,Risk Response Strategies,studied by,Good Risk Management Practice,The project manager should develop risk respon...,"['project level risks', 'develop risk response...","['develop risk response strategy', 'risk respo...",[],Project Risk Management is a valuable componen...,"['project risk management general', 'project r...",[],[],The project manager should develop risk respo...,Project Risk Management is a valuable compone...
4,Scenario Analysis,subclass of,Historical Review,Scenario analysis for risk response planning ...,"['analysis risk response planning', 'scenario ...","['scenario analysis', 'scenario analysis risk']",[],Historical reviews are based on what occurred ...,"['project organization comparable project', 'b...",[],[],Scenario analysis for risk response planning ...,Historical reviews are based on what occurred...
6,Conclusion,subclass of,REFERENCES,The principles of Project Risk Management desc...,"['project risk management provides', 'project ...",[],[],Association for Project Management. 2004. Proj...,"['2004 project risk analysis', 'management 200...",[],[],The principles of Project Risk Management des...,Project risk analysis management PRAM guide s...


In [38]:
relation_df2.to_csv("FINAL_DATASET.csv",index=False)